In [1]:
import torch
import torch.nn as nn

import os
import json
import shutil

from torch.utils.tensorboard import SummaryWriter

##### Import key BCAI ART modules related to:
1. a dataset
2. a model
3. a training environment
4. a training and test loaders
5. an optimizer and a scheduler
6. an attack object
7. a trainer object
8. an evaluator object

In [2]:
from bcai_art.utils_misc import paramdict_to_args, ObjectFromDict, NORM_INF
from bcai_art.datasets_main import create_dataset, get_shaped_tensor, \
    DATASET_TYPE, DATASET_NAME_PARAM, DATASET_ROOT_PARAM, \
    DATASET_MEAN_PARAM, DATASET_STD_PARAM, \
    DATASET_NUM_CHANNELS_PARAM, DATASET_NUM_CLASSES_PARAM, \
    DATASET_LOWER_LIMIT_PARAM, DATASET_UPPER_LIMIT_PARAM
from bcai_art.train import create_optimizer, create_trainer, compute_lr_steps, create_scheduler, \
    TrainEvalEnviron, train, TRAINER_ADV_PLUG_ATTACK
from bcai_art.eval import evaluate, EVALUATOR_NORMAL, EVALUATOR_ATTACK, create_evaluator
from bcai_art.models_main import create_toplevel_model, MODEL_ARCH_ATTR
from bcai_art.attacks import create_attack

##### MNIST dataset
We will eperiment with the MNIST dataset, which will be downloadedautomatically to directory `./data`. Some of our datasets need to be downloaded 
(and optionally processed manually) before they can be used.

In [3]:
DATASET_NAME = 'mnist'
DATASET_ROOT = './data'  # Where to download data

In [4]:
# The most common random seed after 0!!!
# https://twitter.com/jakevdp/status/1247742792861757441
RANDOM_SEED=42

##### Toy model
We use a small, toy, model that works well on MNIST

In [5]:
MODEL_NAME = 'mnistnet'
USE_PRETRAINED = False  # This MNIST model has no pre-trained version

##### We need to create directories to store logs and model snapshots

In [6]:
EXPER_DIR = 'sample_exper_dir/train_pgd_mnist_sample'
SNAPSHOT_DIR = os.path.join(EXPER_DIR, 'snapshots')
LOG_DIR = os.path.join(EXPER_DIR, 'logs')

# EXPER_DIR must go first
for dn in [EXPER_DIR, SNAPSHOT_DIR, LOG_DIR]:
    if os.path.exists(dn):
        shutil.rmtree(dn)
    os.makedirs(dn)

##### We can optionally save logs using Tensorboard

In [7]:
writer = SummaryWriter(log_dir=LOG_DIR)

###### This example runs only on a single GPU
It can run on CPU as well (if you change DEVICE_NAME to 'cpu')

In [8]:
DEVICE_NAME = 'cuda:0'

##### Let us create the dataset object
Dataset parameters returned by this function as well. In this example, they are stored in the variable `dataset_info`

In [9]:
train_set, test_set, \
dataset_info = create_dataset(ObjectFromDict({DATASET_NAME_PARAM: DATASET_NAME,
                                              DATASET_ROOT_PARAM: DATASET_ROOT}))

##### ObjectFromDict explanation
A lot of our API functions receive parameters bundled together in a Python object, where attribute names correspond to parameter names. For convenience, we provide a wrapper object
that converts parameter dictionaries to such objects: `ObjectFromDict`.

##### Let us unpack dataset parameters as we need them for the wrapper model

In [10]:
# Image and video have several input channels
channel_qty = dataset_info[DATASET_NUM_CHANNELS_PARAM]
# Number of classes
class_qty = dataset_info[DATASET_NUM_CLASSES_PARAM]

# Mean, STD and upper/lower limit values are used for normalization and clamping
norm_mean = get_shaped_tensor(dataset_info, DATASET_MEAN_PARAM)
norm_std = get_shaped_tensor(dataset_info, DATASET_STD_PARAM)

upper_limit = get_shaped_tensor(dataset_info, DATASET_UPPER_LIMIT_PARAM)
lower_limit = get_shaped_tensor(dataset_info, DATASET_LOWER_LIMIT_PARAM)

##### Now we can create a (top-level) wrapper model

In [11]:
# A dictionary with optional model arguments (object attributes)
model_args = paramdict_to_args({ MODEL_ARCH_ATTR : MODEL_NAME })

model = create_toplevel_model(num_classes=class_qty,
                              mean = norm_mean,
                              std = norm_std,
                              model_arg_obj=model_args)

number of trainable parameters: 1199882


##### Creating an optimizer and a scheduler

In [12]:
BATCH_SIZE=512
EPOCH_QTY=10

SHUFFLE_TRAIN=True
NUM_DATA_WORKERS=2  # number of processes for the data loader


OPTIMIZER_ARGS_DICT = {"momentum": 0.9, "weight_decay": 0.256}
OPTIM_TYPE='sgd'
# Not used when scheduler is specified
INIT_LR=1e-3

optim_args = paramdict_to_args(OPTIMIZER_ARGS_DICT)
optimizer = create_optimizer(OPTIM_TYPE, model, INIT_LR, optim_args)

# scheduler is optional (can be None)
SCHEDULER_TYPE = 'one_cycle'
SCHEDULER_ARGS = {
    "max_lr": 0.0002,
    "anneal_strategy": "linear",
    "pct_start": 0.2
}
scheduler_args = paramdict_to_args(SCHEDULER_ARGS)


lr_steps = compute_lr_steps(epoch_qty=EPOCH_QTY,
                            train_set=train_set,
                            batch_size=BATCH_SIZE)

if lr_steps > 0:
    scheduler = create_scheduler(optimizer, SCHEDULER_TYPE, lr_steps, scheduler_args)
else:
    print('Not creating the scheduler, because the number of steps is not positive!')
    scheduler = None

##### Creating a training environment object
`TrainEvalEnviron` is a wrapper class for training and evaluation environment that
       encapsulates a model, an optional adversarial perturbation,
       and an optimizer. It also provides a few convenience functions to simplify training,
       including a multi-device/multi-processing training.

In [13]:
train_env = TrainEvalEnviron(DEVICE_NAME,
                             train_set,
                             dataset_info[DATASET_TYPE],
                             BATCH_SIZE,
                             device_qty=1, para_type=None,
                             model=model,
                             train_model=True, # it means that the model will be trained
                             adv_perturb=None,
                             train_attack=False, # the attack is not trained and there're no initial attack weights
                             optimizer=optimizer,
                             lower_limit=lower_limit, upper_limit=upper_limit,
                             log_writer=writer,
                             use_amp=False)

##### Let us create a (PGD) attack object that we use for both training & evaluation

In [14]:
ATTACK_TARGET_ID=None  # Can be used for targeted attack
# Here's a dictionary describing PGD attack parameters
ATTACK_TYPE="pgd"
ATTACK_EPS=0.2
ATTACK_NORM=NORM_INF

ATTACK_ADD_ARGS = {
    "alpha": 0.05,
    "num_iters": 10,
    "restarts": 1,
    "start": "random"
}

# To create an adversarial trainer we need to create an attack object first
add_attack_args = paramdict_to_args(ATTACK_ADD_ARGS)
# PGD attack has no weights file, but for the trainable attack
# one needs to load weights and specify them in the TrainEvalEnviron constructor
trainer_attack_obj = create_attack(ATTACK_TYPE,
                                       inner_attack=None,
                                       target=ATTACK_TARGET_ID,
                                       epsilon=ATTACK_EPS,
                                       norm_name=ATTACK_NORM,
                                       add_args=add_attack_args)

Creating attack: pgd target: None epsilon: 0.2 norm: linf add args: {'alpha': 0.05, 'num_iters': 10, 'restarts': 1, 'start': 'random'}


##### We can now create a trainer object to train the model adversarially

In [15]:
TRAINER_TYPE=TRAINER_ADV_PLUG_ATTACK

trainer = create_trainer(TRAINER_TYPE,
                         train_env,
                         trainer_attack=trainer_attack_obj,
                         trainer_args=None) # No additional arguments for this trainer

##### Creating a data loader

In [16]:
train_loader = torch.utils.data.DataLoader(train_set, pin_memory=True,
                                           batch_size=BATCH_SIZE,
                                           shuffle=SHUFFLE_TRAIN,
                                           drop_last=True, # drop the last incomplete batch!
                                           num_workers=NUM_DATA_WORKERS)

##### Finally, we have all the necessary ingredients to train the model (adversarially)

In [17]:
# Run the training procedure
train(train_loader,
      dataset_info=dataset_info,
      trainer=trainer,
      num_epochs=EPOCH_QTY,
      batch_sync_step=1, # Isn't really used if the number of GPUs == 1
      scheduler=scheduler,
      snapshot_dir=SNAPSHOT_DIR,
      seed=RANDOM_SEED,
      is_master_proc=True,
      print_train_stat=False) # This is only additional debug stat, mostly useless here

lr: 0.000104 avg. loss: 1.4815 avg acc: 0.4846: 100%|█████████████████████████████████████████████████████| 117/117 [00:12<00:00,  9.30it/s]


epoch: 0	 Avg. Loss: 1.4815	 Avg. Accuracy: 0.4846	 Avg. Batch Time: 0.100
total train time: 0.2203 minutes


lr: 0.000200 avg. loss: 0.6580 avg acc: 0.7898: 100%|█████████████████████████████████████████████████████| 117/117 [00:12<00:00,  9.29it/s]


epoch: 1	 Avg. Loss: 0.6580	 Avg. Accuracy: 0.7898	 Avg. Batch Time: 0.101
total train time: 0.4310 minutes


lr: 0.000175 avg. loss: 0.4704 avg acc: 0.8531: 100%|█████████████████████████████████████████████████████| 117/117 [00:12<00:00,  9.21it/s]


...

epoch: 7	 Avg. Loss: 0.2819	 Avg. Accuracy: 0.9148	 Avg. Batch Time: 0.101
total train time: 1.7018 minutes


lr: 0.000025 avg. loss: 0.2636 avg acc: 0.9188: 100%|█████████████████████████████████████████████████████| 117/117 [00:12<00:00,  9.34it/s]


epoch: 8	 Avg. Loss: 0.2636	 Avg. Accuracy: 0.9188	 Avg. Batch Time: 0.101
total train time: 1.9114 minutes


lr: -0.000000 avg. loss: 0.2495 avg acc: 0.9248: 100%|████████████████████████████████████████████████████| 117/117 [00:12<00:00,  9.31it/s]


epoch: 9	 Avg. Loss: 0.2495	 Avg. Accuracy: 0.9248	 Avg. Batch Time: 0.101
total train time: 2.1216 minutes


##### Creating an evaluation environment object
Similar to training, we create the object of the type `TrainEvalEnviron`

In [18]:
# Finally evaluate the model, this requires a separate environment:
eval_train_env = TrainEvalEnviron(DEVICE_NAME,
                                  test_set,
                                  dataset_info[DATASET_TYPE],
                                  BATCH_SIZE,
                                  device_qty=1, para_type=None,
                                  model=model, train_model=False, # no model training
                                  adv_perturb=None, train_attack=False, # no attack training
                                  optimizer=None,
                                  lower_limit=lower_limit, upper_limit=upper_limit,
                                  log_writer=writer, use_amp=False)

##### Let us create a test loader and to evaluate the model
The model is going to be tested under normal circumstances as well as under the attack

In [19]:
test_loader = torch.utils.data.DataLoader(test_set, pin_memory=True,
                                           batch_size=BATCH_SIZE,
                                           shuffle=False,
                                           drop_last=False,
                                           num_workers=NUM_DATA_WORKERS)

##### Evaluation on clean, i.e., unmodified data

In [20]:
EVAL_ADD_ARGS_NORMAL = {}
eval_add_args_normal = paramdict_to_args(EVAL_ADD_ARGS_NORMAL)

# We run two evaluations here: normal & PGD adversary attack
evaluator_obj_normal = create_evaluator(EVALUATOR_NORMAL, 
                                        eval_train_env, 
                                        eval_add_args_normal, 
                                        eval_attack_obj=None)

stat = evaluate(test_loader, 'normal evaluator', evaluator_obj_normal)
print(json.dumps(stat, indent=4))

{}


avg. loss: 0.0277 avg acc: 0.9912: 100%|████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.54it/s]


Average batch time: 0.005
loss 0.0277
accuracy 0.9912
{
    "batch_time": 0.004989314079284668,
    "loss": 0.027656947374343873,
    "accuracy": 0.9912
}


##### Evaluation of the model under attack
Note that we use the type of the attack as we used for training, but we can surely use a different type of attack

In [21]:
SAMPLE_QTY=20 # Number of perturbed images to save

EVAL_ADD_ARGS_ATTACK = {'sample_qty' : SAMPLE_QTY}
eval_add_args_attack = paramdict_to_args(EVAL_ADD_ARGS_ATTACK)
eval_attack_obj = trainer_attack_obj # We are reusing the same attack object, but we can create a different one too
evaluator_obj_pgd = create_evaluator(EVALUATOR_ATTACK, 
                                     eval_train_env, 
                                     eval_add_args_attack, 
                                     eval_attack_obj)

{'sample_qty': 20}


##### As we can see, the model performs worse under attack:

In [22]:
stat = evaluate(test_loader, 'PGD evaluator', evaluator_obj_pgd)
print(json.dumps(stat, indent=4))

avg. loss: 0.1423 avg acc: 0.9558: 100%|████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  9.07it/s]


Average batch time: 0.095
loss 0.1423
accuracy 0.9558
attack_success 0.0360
{
    "batch_time": 0.09535449743270874,
    "loss": 0.14231307804584503,
    "accuracy": 0.9558,
    "attack_success": 0.036
}


##### In the end, we need to close the Tensorboard log writer

In [23]:
writer.close()